# 🛠️ Agent Tools with Microsoft Agent Framework (Python)

## 📋 What You'll Learn

This notebook demonstrates how to enhance agents with specialized tools using Azure AI Foundry. You'll see how to add capabilities like file search, code execution, web search, vision processing, and MCP tool integration.

**Tools Covered:**
- 📄 **File Search (RAG)**: Query uploaded documents with vector search
- 💻 **Code Interpreter**: Execute Python code to solve problems
- 🌐 **Web Search (Bing)**: Real-time web information retrieval
- 👁️ **Vision**: Process and analyze images
- 🔌 **MCP Tools**: Connect to external services via Model Context Protocol

## ⚙️ Setup Requirements

### 1. **Docker & Dev Container**
- Install Docker on your machine
- Open this project in the dev container (VS Code will prompt you)

### 2. **Workspace Configuration**
- Open the workspace: `File > Open Workspace from File` → select `workspace.code-workspace`
- This sets up the correct Python environment for each project folder

### 3. **Environment Variables**
1. Copy `.env.example` to `.env` in the `agent-framework-samples` folder:
   ```bash
   cp .env.example .env
   ```
2. Fill in your credentials:
   ```env
   # Azure AI Foundry (Required for all examples)
   AZURE_AI_PROJECT_ENDPOINT=your_project_endpoint
   AZURE_AI_MODEL_DEPLOYMENT_NAME=your_model_deployment
   
   # Bing Search (Required for Web Search example)
   BING_CONNECTION_ID=your_bing_connection_id
   ```

### 4. **Azure AI Foundry Setup**
- Create an Azure AI Foundry project
- Deploy a model (e.g., gpt-4o, gpt-4o-mini)
- For Bing Search: Create a Bing Search connection in your project

## 🚀 What You'll Build

Below you'll find code samples demonstrating:

1. **RAG with File Search**: Upload documents and query them with semantic search
2. **Code Execution**: Let agents write and run Python code (Fibonacci example)
3. **Web Research**: Search the web and get real-time information
4. **Vision Processing**: Analyze images and extract information (furniture example)
5. **MCP Integration**: Two ways to connect external tools via MCP

Each example shows the complete flow: creating the agent, configuring tools, and running queries.

Let's explore agent capabilities! 🌟

In [1]:
import os

from azure.identity.aio import AzureCliCredential, DefaultAzureCredential
from azure.ai.projects.aio import AIProjectClient
from dotenv import load_dotenv
import base64

from agent_framework.azure import AzureAIAgentClient
from azure.ai.agents.models import (
    FilePurpose,
    FileSearchTool, 
    CodeInterpreterTool, 
    BingGroundingTool)
from agent_framework import (
    ChatAgent,
    HostedFileSearchTool,
    HostedVectorStoreContent, 
    HostedCodeInterpreterTool, 
    HostedWebSearchTool,
    ChatMessage,
    DataContent,
    Role, 
    TextContent,
    MCPStreamableHTTPTool)

from azure.ai.projects.aio import AIProjectClient
load_dotenv()

True

In [10]:
async def create_vector_store(client: AIProjectClient) -> tuple[str, HostedVectorStoreContent]:
    """Create a vector store with sample documents."""
    file_path = 'files/document.md'
    file = await client.agents.files.upload_and_poll(file_path=file_path, purpose="assistants")
    print(f"Uploaded file, file ID: {file.id}")


    vector_store = await client.agents.vector_stores.create_and_poll(file_ids=[file.id], name="corporate_insurance")

    print(f"Created vector store, ID: {vector_store.id}")


    return file.id, HostedVectorStoreContent(vector_store_id=vector_store.id)

async with (
        DefaultAzureCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as project_client,
    ):
        file_id, vector_store = await create_vector_store(project_client)
        file_search = FileSearchTool(vector_store_ids=[vector_store.vector_store_id])
        
        created_agent = await project_client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
            name="PythonRAGAgent",
            instructions="""
                    You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

                    - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
                    "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
                    - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
                    - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
                    - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
                    - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

                    You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
                    """,
            tools = file_search.definitions,
            tool_resources= file_search.resources
        )
        chat_client=AzureAIAgentClient(agent_id=created_agent.id, async_credential = credential)


        async with ChatAgent(
                # passing in the client is optional here, so if you take the agent_id from the portal
                # you can use it directly without the two lines above.
                chat_client=chat_client,
            ) as agent:
                    

            print("Agent created. You can now ask questions about the uploaded document.")

            query = "Can you explain Contoso's travel insurance coverage?"

            async for chunk in agent.run_stream(query, tools=HostedFileSearchTool(inputs=vector_store)):

                if chunk.text:
                    print(chunk.text, end="", flush=True)

Uploaded file, file ID: assistant-BKYn46P2oSc53gUQcuF1cj
Created vector store, ID: vs_nx2sjSsL8xNO31IXjuERwyw0
Agent created. You can now ask questions about the uploaded document.
Contoso's travel insurance coverage provides the following benefits:

- Medical coverage of up to $100,000 while traveling for business purposes.
- The policy covers the cost of emergency medical treatment, hospital stays, and evacuation if necessary.
- Lost luggage is covered up to $2,000 per trip.
- The insurance also provides 24/7 travel assistance services, which can help with things like lost passports or arranging transport in case of illness.

These benefits are available to all full-time employees traveling on approved business trips. Coverage is effective from the time of departure until return to the home location and does not extend to personal travel or vacation extensions outside of business purposes .

In [7]:
AgentName = "Coding-Agent"
AgentInstructions = "You are an AI assistant that helps people find information."

code = CodeInterpreterTool()

async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):
        created_agent = await client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
            instructions=AgentInstructions,
            name=AgentName,
            tools=code.definitions
        )
        chat_client=AzureAIAgentClient(project_client=client, agent_id=created_agent.id)

        async with ChatAgent(
                # passing in the client is optional here, so if you take the agent_id from the portal
                # you can use it directly without the two lines above.
                chat_client=chat_client,
                tools=HostedCodeInterpreterTool()
            ) as agent:
            message = "Use code to determine the values in the Fibonacci sequence that that are less then the value of 101?"
            # response = agent.run_stream(message)
            first_result = await agent.run(message)
            
            print(f"Agent: {first_result.text}")

Agent: Certainly! Here is some Python code that will generate all Fibonacci numbers less than 101:

```python
# List to hold Fibonacci numbers
fibonacci_numbers = []

# Starting values
a, b = 0, 1

while a < 101:
    fibonacci_numbers.append(a)
    a, b = b, a + b

print(fibonacci_numbers)
```

Let's execute this to get the Fibonacci numbers less than 101.The values in the Fibonacci sequence that are less than 101 are:

\[ 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89 \]


In [2]:
# Bing Grounding Tool Example 

conn_id = os.environ["BING_CONNECTION_ID"]  # Ensure the BING_CONNECTION_NAME environment variable is set

# Initialize the Bing Grounding tool
bing = BingGroundingTool(connection_id=conn_id)
print(bing)

async with (
        DefaultAzureCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):

        created_agent = await client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
            instructions="You are a helpful assistant that can help with web search questions.",
            name="WebResearchAgent",
            tools=bing.definitions
        )
        chat_client=AzureAIAgentClient(project_client=client, agent_id=created_agent.id)

        async with ChatAgent(
                # passing in the client is optional here, so if you take the agent_id from the portal
                # you can use it directly without the two lines above.
                chat_client=chat_client,
                tools=HostedWebSearchTool()
            ) as agent:
            message = "Could you please describe the workshop according to the link https://github.com/SoTrx/GHCAgentWorkshop?"
            # response = agent.run_stream(message)
            first_result = await agent.run(message)
            
            print(f"Agent: {first_result.text}")

Agent: The GHCAgentWorkshop, also known as the GitHub Copilot Agent Workshop, is designed to help participants create Copilot applications with zero code. The workshop focuses on leveraging AI to enable anyone to manage projects like a project manager, using the new Agent mode in GitHub Copilot Chat. The workshop includes a series of practical labs:

1. **Lab 1 - RAG Application**: Learn the capabilities of GitHub Copilot, use the Agent mode in Copilot Edit, create a Copilot application frontend, and invoke GitHub Models via Copilot Agent.
2. **Lab 2 - Stock Analysis**: Generate simulated stock data using Copilot Agent, build a stock analysis UI, and construct both frontend and backend features.
3. **Lab 3 - Experience GPT-5 & Application Deployment**: Get familiar with the GPT-5 model within Copilot, create an interactive lottery wheel application, and deploy your application using GitHub Pages.

Participants need a GitHub account, basic tools like Python (3.10.8+), Visual Studio Code

In [4]:
AgentName = "Vision-Agent"
AgentInstructions = "You are my furniture sales consultant, you can find different furniture elements from the pictures and give me a purchase suggestion"

async with (
        DefaultAzureCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):
        created_agent = await client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
            instructions=AgentInstructions,
            name=AgentName
        )
        chat_client=AzureAIAgentClient(project_client=client, agent_id=created_agent.id)

        async with ChatAgent(
                # passing in the client is optional here, so if you take the agent_id from the portal
                # you can use it directly without the two lines above.
                chat_client=chat_client
            ) as agent:
    

            image_path = "files/home.png"
            with open(image_path, "rb") as image_file:
                image_b64 = base64.b64encode(image_file.read()).decode()
            image_uri = f"data:image/png;base64,{image_b64}"
            message = ChatMessage(
                    role=Role.USER,
                    contents=[
                        TextContent(text="Please find the relevant furniture according to the image and give the corresponding price for each piece of furniture"),
                        DataContent(uri=image_uri, media_type="image/png")
                    ]
            )
            # response = agent.run_stream(message)
            first_result = await agent.run(message)
            
            print(f"Agent: {first_result.text}")

Agent: Absolutely! Here is a breakdown of the key furniture pieces visible in the image, along with estimated prices for each item based on popular online retailers and contemporary furniture stores:

---

### 1. **Modern Fabric Sofa**
- **Description:** A long, modern white/cream sofa with multiple blue and white throw pillows.
- **Estimated Price:** \$900 - \$1,500

### 2. **Mid-Century Coffee Table**
- **Description:** Oval-shaped coffee table with angled wooden legs and a decorative centerpiece.
- **Estimated Price:** \$180 - \$400

### 3. **Wood TV Console**
- **Description:** Low, wide wood entertainment unit with drawers and open shelves.
- **Estimated Price:** \$350 - \$800

### 4. **Accent Chair**
- **Description:** Dark blue, modern chair with a unique, enveloping backrest.
- **Estimated Price:** \$350 - \$700

### 5. **Side Table (by accent chair)**
- **Description:** Modern, tripod-style round side table with a plant and decor.
- **Estimated Price:** \$90 - \$200

### 6. **

# MCP Tool calling : Learn Microsoft MCP Tool

In [5]:
# Calling the MCP Tool from the agent's context (configuration time) 

async with (
    DefaultAzureCredential() as credential,
    AzureAIAgentClient(async_credential=credential).create_agent(
        name="DocsAgent",
        instructions="You are a helpful assistant that can help with microsoft documentation questions.",
        tools=MCPStreamableHTTPTool(  # Tools defined at agent creation
            name="Microsoft Learn MCP",
            url="https://learn.microsoft.com/api/mcp",
        ),
    ) as agent,
):
    query = "What is Microsoft Semantic Kernel?"
    print(f"User: {query}")
    result = await agent.run(query)
    print(f"{agent.name}: {result}\n")

User: What is Microsoft Semantic Kernel?


[2025-11-28 12:37:51 - /workspaces/graphrag-evaluator/agent-framework-samples/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


DocsAgent: Microsoft Semantic Kernel is an open-source SDK for integrating AI capabilities, such as large language models (LLMs) and copilots, into applications. It provides tools to help developers orchestrate AI services, manage prompts, plug in external data sources, and build advanced agentic applications (like chatbots or digital assistants) in a modular way across .NET, Python, and Java.

Key concepts of Semantic Kernel include:

- Kernel: The central component that manages resources, AI models, and plugins, allowing integration of both native code and AI services.
- Plugins & Connectors: Tools to connect with APIs, enrich your AI with external functions, or integrate with sources like Microsoft Graph or OpenAI.
- Agents: AI-powered logical entities (e.g., chatbots, copilots) that can automate tasks, answer questions, or orchestrate processes.
- Planning & Orchestration: Built-in mechanisms for the AI to plan and execute multi-step processes or workflows on the fly.

Semantic Ker

In [6]:
# Setting the MCP Tool during the runtime agent's execution  

async with (
    DefaultAzureCredential() as credential,
    MCPStreamableHTTPTool(
        name="Microsoft Learn MCP",
        url="https://learn.microsoft.com/api/mcp",
    ) as mcp_server,
    ChatAgent(
        chat_client=AzureAIAgentClient(async_credential=credential),
        name="DocsAgent",
        instructions="You are a helpful assistant that can help with microsoft documentation questions.",
    ) as agent,
):
    query = "What is Microsoft Semantic Kernel?"
    print(f"User: {query}")
    result = await agent.run(query, tools=mcp_server)
    print(f"{agent.name}: {result}\n")

User: What is Microsoft Semantic Kernel?


[2025-11-28 12:38:24 - /workspaces/graphrag-evaluator/agent-framework-samples/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


DocsAgent: Microsoft Semantic Kernel is an open-source SDK (Software Development Kit) that enables developers to integrate and orchestrate AI capabilities—especially large language models (LLMs) like OpenAI, Azure OpenAI, and others—into their .NET, Python, or Java applications.

### What does Semantic Kernel do?
- **Central Orchestrator:** It serves as the "kernel," a central component that manages all the services (like LLMs) and plugins (custom code and logic) necessary to build your AI application.
- **Flexible Integration:** You can add different AI services, build workflows that combine AI with your own code, and add enterprise features like logging.
- **Build Agents & Copilots:** Enables creation of sophisticated AI agents, from simple chatbots to complex copilots and autonomous agents, even for users without deep AI expertise.
- **Prompt Engineering:** Supports advanced prompt development, including prompt templating, chaining, and memory.
- **Enterprise Ready:** Suited for cre